# Boto3 & LocalStudio

![Boto3 & LocalStudio image](img/s3-10.png)


## 🏠 What is localstack?

**Localstack** is a platform that provides a local version of several cloud services, allowing you to simulate a development environment with AWS services. This allows you to debug and refine your code before deploying it to a production environment. For this reason, Localstack is a valuable tool for emulating essential AWS services such as object storage and message queues, among others.

Also, **Localstack** serves as an effective tool for learning to implement and deploy services using a Docker container without the need for an AWS account or the use of your credit card. 
In this tutorial, we create a Localstack container to implement the main functionalities of S3 services.

---

## What is boto3?

**`Boto3`** is a 🐍 Python library that allows the integration with AWS services, facilitating various tasks such as creation, management, and configuration of these services.

There are two primary implementations within Boto3: 
* **Resource implementation**: provides a higher-level, object-oriented interface, abstracting away low-level details and offering simplified interactions with AWS services. 
* **Client implementation**: offers a lower-level, service-oriented interface, providing more granular control and flexibility for interacting with AWS services directly.


---

## Prerequisites
Before you begin, ensure that you have the following installed:

* 🐳 Docker
* 🐙 Docker Compose


---

### 🚀 Build and run the Docker Compose environment

#### 1. Clone the repository
 ```bash
   git clone https://github.com/r0mymendez/LocalStack-boto3.git
   cd LocalStack-boto3
```
#### 2. Build an run the docker compose 
  
`docker-compose -f docker-compose.yaml up --build`

---

### 🚀 Using LocalStack with Boto3: A Step-by-Step Guide
### 🛠️ Install Boto3

```!pip install boto3```




In [2]:
import boto3
import json 
import requests
import pandas as pd
from datetime import datetime
import io
import os

### 🛠️ Create a session using the localstack endpoint
The following code snippet initializes a client for accessing the S3 service using the LocalStack endpoint.

In [3]:

s3 = boto3.client(
    service_name='s3',
    aws_access_key_id='test',
    aws_secret_access_key='test',
    endpoint_url='http://localhost:4566',
)

### 🛠️ Create new buckets
Below is the code snippet to create new buckets using the Boto3 library

In [4]:
# create buckets
bucket_name_news = 'news'
bucket_name_config = 'news-config'

s3.create_bucket(Bucket=bucket_name_news)
s3.create_bucket(Bucket=bucket_name_config)

{'ResponseMetadata': {'RequestId': '0aa0f792-c5e3-4167-b5c2-bcb930198276',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml',
   'access-control-allow-origin': '*',
   'access-control-allow-methods': 'HEAD,GET,PUT,POST,DELETE,OPTIONS,PATCH',
   'access-control-allow-headers': 'authorization,cache-control,content-length,content-md5,content-type,etag,location,x-amz-acl,x-amz-content-sha256,x-amz-date,x-amz-request-id,x-amz-security-token,x-amz-tagging,x-amz-target,x-amz-user-agent,x-amz-version-id,x-amzn-requestid,x-localstack-target,amz-sdk-invocation-id,amz-sdk-request',
   'access-control-expose-headers': 'etag,x-amz-version-id',
   'vary': 'Origin',
   'location': '/news-config',
   'x-amz-request-id': '0aa0f792-c5e3-4167-b5c2-bcb930198276',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   '

### 📋 List all buckets
After creating a bucket, you can use the following code to list all the buckets available at your endpoint.

In [5]:
# List all buckets
response = s3.list_buckets()
response['Buckets']
# pd.json_normalize(response['Buckets'])

[{'Name': 'news',
  'CreationDate': datetime.datetime(2024, 4, 13, 8, 36, 9, tzinfo=tzutc())},
 {'Name': 'news-config',
  'CreationDate': datetime.datetime(2024, 4, 13, 8, 36, 9, tzinfo=tzutc())}]

### 📤 Upload the JSON file to s3
Once we extract data from the API to gather information about news topics, the following code generates a JSON file and uploads it to the S3 bucket previously created.

In [6]:
# invoke the config news
url = 'https://ok.surf/api/v1/cors/news-section-names' 
response = requests.get(url)
if response.status_code==200:
    data = response.json()
    # ad json file to s3
    print('data', data)
    # upload the data to s3
    s3.put_object(Bucket=bucket_name_config, Key='news-section/data_config.json', Body=json.dumps(data))



data ['US', 'World', 'Business', 'Technology', 'Entertainment', 'Sports', 'Science', 'Health']


### 📋 List all objects
Now, let's list all the objects stored in our bucket. Since we might have stored a JSON file in the previous step, we'll include code to retrieve all objects from the bucket.

In [7]:
def list_objects(bucket_name):
    response = s3.list_objects(Bucket=bucket_name)
    return pd.json_normalize(response['Contents'])

In [8]:
# list all objects in the bucket
list_objects(bucket_name=bucket_name_config)

,Key,LastModified,ETag,Size,StorageClass,Owner.DisplayName,Owner.ID
0,news-section/data_config.json,2024-04-13 08:36:21+00:00,"""d61029b184d21dae1febcb46062216d3""",89,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


In [9]:
obj = s3.get_object(Bucket=bucket_name_config, Key='news-section/data_config.json')
# data = pd.read_csv(obj['Body'])
# 
# data
pd.read_json(obj['Body'])

,0
0,US
1,World
2,Business
3,Technology
4,Entertainment
5,Sports
6,Science
7,Health


###  📄 Upload multiple CSV files to s3 
In the following code snippet, we will request another method from the API to extract news for each topic. Subsequently, we will create different folders in the bucket to save CSV files containing the news for each topic. This code enables you to save multiple files in the same bucket while organizing them into folders based on the topic and the date of the data request.

In [10]:
# Request the news feed API Method
url = 'https://ok.surf/api/v1/news-feed' 
response = requests.get(url)
if response.status_code==200:
    data = response.json()

# Add the json file to s3
folder_dt =  f'dt={datetime.now().strftime("%Y%m%d")}'

for item in data.keys():
    tmp = pd.json_normalize(data[item])
    tmp['section'] = item   
    tmp['download_date'] = datetime.now()
    tmp['date'] = pd.to_datetime(tmp['download_date']).dt.date
    path = f"s3://{bucket_name_news}/{item}/{folder_dt}/data_{item}_news.csv"

    # upload multiple files to s3
    bytes_io = io.BytesIO()
    tmp.to_csv(bytes_io, index=False)
    bytes_io.seek(0)
    s3.put_object(Bucket=bucket_name_news, Key=path, Body=bytes_io)



In [11]:
# list all objects in the bucket
list_objects(bucket_name=bucket_name_news)


,Key,LastModified,ETag,Size,StorageClass,Owner.DisplayName,Owner.ID
0,s3://news/Business/dt=20240413/data_Business_n...,2024-04-13 08:36:35+00:00,"""dde88c71110a9c2702dea3f0664104ad""",44450,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
1,s3://news/Entertainment/dt=20240413/data_Enter...,2024-04-13 08:36:35+00:00,"""6208fcf0acc5fd4194e104aa75d6f0ab""",20884,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
2,s3://news/Health/dt=20240413/data_Health_news.csv,2024-04-13 08:36:35+00:00,"""7ee0751cd44a6fd2288d9e60c437ed18""",47526,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
3,s3://news/Science/dt=20240413/data_Science_new...,2024-04-13 08:36:35+00:00,"""ddce3cced04a91878d95f4be0c14d833""",43407,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
4,s3://news/Sports/dt=20240413/data_Sports_news.csv,2024-04-13 08:36:35+00:00,"""83c357ae6a767471f5497f2428853249""",30624,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
5,s3://news/Technology/dt=20240413/data_Technolo...,2024-04-13 08:36:35+00:00,"""3ef519c3d6d503512ffb055a46188a3f""",31218,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
6,s3://news/US/dt=20240413/data_US_news.csv,2024-04-13 08:36:35+00:00,"""59c27a6bc7bba8a7de2e2d0dd6bc6871""",37360,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
7,s3://news/World/dt=20240413/data_World_news.csv,2024-04-13 08:36:36+00:00,"""17472b76f0644abc5726db0ae4b6dfb4""",30499,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


###  📄 Read csv file from s3 
In this section, we aim to read a file containing news about technology topics from S3. To accomplish this, we first retrieve the name of the file in the bucket. Then, we read this file and print the contents as a pandas dataframe.

In [12]:
# Get the technology file
files = list_objects(bucket_name=bucket_name_news)
technology_file = files[files['Key'].str.find('Technology')>=0]['Key'].values[0]
print('file_name',technology_file)

file_name s3://news/Technology/dt=20240413/data_Technology_news.csv


In [13]:
# get the file from s3 using boto3
obj = s3.get_object(Bucket=bucket_name_news, Key=technology_file)
data_tech = pd.read_csv(obj['Body'])

data_tech

,link,og,source,source_icon,title,section,download_date,date
0,https://news.google.com/articles/CBMiQmh0dHBzO...,https://encrypted-tbn3.gstatic.com/images?q=tb...,9to5Google,https://encrypted-tbn3.gstatic.com/faviconV2?u...,Android 15 Beta 1 and the curious case of 'Pix...,Technology,2024-04-13 11:36:35.837480,2024-04-13
1,https://news.google.com/articles/CBMiOGh0dHBzO...,https://encrypted-tbn3.gstatic.com/images?q=tb...,9to5Mac,https://encrypted-tbn0.gstatic.com/faviconV2?u...,Four things to expect from the next-generation...,Technology,2024-04-13 11:36:35.837480,2024-04-13
2,https://news.google.com/articles/CBMiWmh0dHBzO...,https://encrypted-tbn2.gstatic.com/images?q=tb...,Business Insider,https://encrypted-tbn3.gstatic.com/faviconV2?u...,Humane engineer admits Ai pin can be 'frustrat...,Technology,2024-04-13 11:36:35.837480,2024-04-13
3,https://news.google.com/articles/CBMiPGh0dHBzO...,https://encrypted-tbn3.gstatic.com/images?q=tb...,The Register,https://encrypted-tbn1.gstatic.com/faviconV2?u...,Google One VPN axed for everyone but Pixel loy...,Technology,2024-04-13 11:36:35.837480,2024-04-13
4,https://news.google.com/articles/CBMiT2h0dHBzO...,https://encrypted-tbn1.gstatic.com/images?q=tb...,Yahoo Finance,https://encrypted-tbn1.gstatic.com/faviconV2?u...,US lawmakers angry after Huawei unveils laptop...,Technology,2024-04-13 11:36:35.837480,2024-04-13
5,https://news.google.com/articles/CBMiV2h0dHBzO...,https://encrypted-tbn2.gstatic.com/images?q=tb...,Variety,https://encrypted-tbn1.gstatic.com/faviconV2?u...,‘Fallout 76’ Is Now Free For New Prime Members...,Technology,2024-04-13 11:36:35.837480,2024-04-13
6,https://news.google.com/articles/CBMiSWh0dHBzO...,https://encrypted-tbn2.gstatic.com/images?q=tb...,Investor's Business Daily,https://encrypted-tbn0.gstatic.com/faviconV2?u...,Apple Stock Rises On AI Mac Report,Technology,2024-04-13 11:36:35.837480,2024-04-13
7,https://news.google.com/articles/CBMiY2h0dHBzO...,https://encrypted-tbn2.gstatic.com/images?q=tb...,The Verge,https://encrypted-tbn2.gstatic.com/faviconV2?u...,Galaxy AI features are coming to last-gen Sams...,Technology,2024-04-13 11:36:35.837480,2024-04-13
8,https://news.google.com/articles/CBMiamh0dHBzO...,https://encrypted-tbn2.gstatic.com/images?q=tb...,The Information,https://encrypted-tbn3.gstatic.com/faviconV2?u...,"At Google's Cloud Conference, Lofty AI Visions...",Technology,2024-04-13 11:36:35.837480,2024-04-13
9,https://news.google.com/articles/CBMiWWh0dHBzO...,https://encrypted-tbn1.gstatic.com/images?q=tb...,9to5Google,https://encrypted-tbn3.gstatic.com/faviconV2?u...,Microsoft is sticking ads in the Windows 11 St...,Technology,2024-04-13 11:36:35.837480,2024-04-13


 ### 🏷️ Add tags to the bucket
 When creating a resource in the cloud, it is considered a best practice to add tags for organizing resources, controlling costs, or applying security policies based on these labels. The following code demonstrates how to add tags to a bucket using a method from the boto3 library.

In [14]:
s3.put_bucket_tagging(
    Bucket=bucket_name_news,
    Tagging={
        'TagSet': [
            {
                'Key': 'Environment',
                'Value': 'Test'
            },
            {
                'Key': 'Project',
                'Value': 'Localstack+Boto3'
            }
        ]
    }
)

{'ResponseMetadata': {'RequestId': '88d41332-3365-4d7b-be42-6a2104a2aea7',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'content-type': 'application/xml',
   'x-amz-request-id': '88d41332-3365-4d7b-be42-6a2104a2aea7',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   'date': 'Sat, 13 Apr 2024 08:36:49 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [15]:
# get the tagging
pd.json_normalize(s3.get_bucket_tagging(Bucket=bucket_name_news)['TagSet'])

,Key,Value
0,Environment,Test
1,Project,Localstack+Boto3


In [16]:
# list_objects(bucket_name_news)
pd.json_normalize(s3.list_objects(Bucket=bucket_name_news)['Contents'])

,Key,LastModified,ETag,Size,StorageClass,Owner.DisplayName,Owner.ID
0,s3://news/Business/dt=20240413/data_Business_n...,2024-04-13 08:36:35+00:00,"""dde88c71110a9c2702dea3f0664104ad""",44450,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
1,s3://news/Entertainment/dt=20240413/data_Enter...,2024-04-13 08:36:35+00:00,"""6208fcf0acc5fd4194e104aa75d6f0ab""",20884,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
2,s3://news/Health/dt=20240413/data_Health_news.csv,2024-04-13 08:36:35+00:00,"""7ee0751cd44a6fd2288d9e60c437ed18""",47526,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
3,s3://news/Science/dt=20240413/data_Science_new...,2024-04-13 08:36:35+00:00,"""ddce3cced04a91878d95f4be0c14d833""",43407,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
4,s3://news/Sports/dt=20240413/data_Sports_news.csv,2024-04-13 08:36:35+00:00,"""83c357ae6a767471f5497f2428853249""",30624,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
5,s3://news/Technology/dt=20240413/data_Technolo...,2024-04-13 08:36:35+00:00,"""3ef519c3d6d503512ffb055a46188a3f""",31218,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
6,s3://news/US/dt=20240413/data_US_news.csv,2024-04-13 08:36:35+00:00,"""59c27a6bc7bba8a7de2e2d0dd6bc6871""",37360,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
7,s3://news/World/dt=20240413/data_World_news.csv,2024-04-13 08:36:36+00:00,"""17472b76f0644abc5726db0ae4b6dfb4""",30499,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


### 🔄 Versioning in the bucket
Another good practice to apply is enabling versioning for your bucket. Versioning provides a way to recover and keep different versions of the same object. In the following code, we will create a file with the inventory of objects in the bucket and save the file twice. 

In [17]:

# allow versioning in the bucket
s3.put_bucket_versioning(
    Bucket=bucket_name_news,
    VersioningConfiguration={
        'Status': 'Enabled'
    }
)



{'ResponseMetadata': {'RequestId': '39d84f34-f2ad-49c8-9653-90b8b7d59bed',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml',
   'x-amz-request-id': '39d84f34-f2ad-49c8-9653-90b8b7d59bed',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   'content-length': '0',
   'date': 'Sat, 13 Apr 2024 08:37:06 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [18]:
# Add new file to the bucket

# file name
file_name = 'inventory.csv'

# list all objects in the bucket
files = list_objects(bucket_name=bucket_name_news)
bytes_io = io.BytesIO()
files.to_csv(bytes_io, index=False)
bytes_io.seek(0)
# upload the data to s3
s3.put_object(Bucket=bucket_name_news, Key=file_name, Body=bytes_io)



{'ResponseMetadata': {'RequestId': 'e4b1e2ce-d9e2-46d6-bb1b-4e02811b4c04',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml',
   'etag': '"435141e1d042edf8e083b940b134d86b"',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-version-id': 'ipACwABj1Nsx1qr35dK1Rw',
   'x-amz-request-id': 'e4b1e2ce-d9e2-46d6-bb1b-4e02811b4c04',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   'content-length': '0',
   'date': 'Sat, 13 Apr 2024 08:37:09 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0},
 'ETag': '"435141e1d042edf8e083b940b134d86b"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'ipACwABj1Nsx1qr35dK1Rw'}

In [19]:
pd.read_csv(s3.get_object(Bucket=bucket_name_news, Key='inventory.csv')['Body'])

,Key,LastModified,ETag,Size,StorageClass,Owner.DisplayName,Owner.ID
0,s3://news/Business/dt=20240413/data_Business_n...,2024-04-13 08:36:35+00:00,"""dde88c71110a9c2702dea3f0664104ad""",44450,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
1,s3://news/Entertainment/dt=20240413/data_Enter...,2024-04-13 08:36:35+00:00,"""6208fcf0acc5fd4194e104aa75d6f0ab""",20884,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
2,s3://news/Health/dt=20240413/data_Health_news.csv,2024-04-13 08:36:35+00:00,"""7ee0751cd44a6fd2288d9e60c437ed18""",47526,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
3,s3://news/Science/dt=20240413/data_Science_new...,2024-04-13 08:36:35+00:00,"""ddce3cced04a91878d95f4be0c14d833""",43407,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
4,s3://news/Sports/dt=20240413/data_Sports_news.csv,2024-04-13 08:36:35+00:00,"""83c357ae6a767471f5497f2428853249""",30624,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
5,s3://news/Technology/dt=20240413/data_Technolo...,2024-04-13 08:36:35+00:00,"""3ef519c3d6d503512ffb055a46188a3f""",31218,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
6,s3://news/US/dt=20240413/data_US_news.csv,2024-04-13 08:36:35+00:00,"""59c27a6bc7bba8a7de2e2d0dd6bc6871""",37360,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
7,s3://news/World/dt=20240413/data_World_news.csv,2024-04-13 08:36:36+00:00,"""17472b76f0644abc5726db0ae4b6dfb4""",30499,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


In [20]:
# add again the same file
s3.put_object(Bucket=bucket_name_news, Key=file_name, Body=bytes_io)

{'ResponseMetadata': {'RequestId': '2b4b542b-99d9-478a-8ed5-7171b4767fbf',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-version-id': 'EZnUkQRyspDf1L47CUpncQ',
   'x-amz-request-id': '2b4b542b-99d9-478a-8ed5-7171b4767fbf',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   'content-length': '0',
   'date': 'Sat, 13 Apr 2024 08:37:17 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'EZnUkQRyspDf1L47CUpncQ'}

In [29]:
# List all the version of the object
versions = s3.list_object_versions(Bucket=bucket_name_news, Prefix=file_name)

pd.json_normalize(versions['Versions'])

,ETag,Size,StorageClass,Key,VersionId,IsLatest,LastModified,Owner.DisplayName,Owner.ID
0,"""d41d8cd98f00b204e9800998ecf8427e""",0,STANDARD,inventory.csv,EZnUkQRyspDf1L47CUpncQ,True,2024-04-13 08:37:17+00:00,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
1,"""435141e1d042edf8e083b940b134d86b""",1718,STANDARD,inventory.csv,ipACwABj1Nsx1qr35dK1Rw,False,2024-04-13 08:37:09+00:00,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


In [34]:
file = open('20240331_163729.jpg', 'rb')
s3.put_object(Bucket=bucket_name_news, Key=file.name, Body=file)

{'ResponseMetadata': {'RequestId': '2f24fc24-86db-4201-a974-169473d57f32',
  'HostId': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/xml',
   'etag': '"d2435bcd89aa4592e75f8555dddcffd4"',
   'x-amz-server-side-encryption': 'AES256',
   'x-amz-version-id': 'OFZ-Ddzgf6apYdhL9tiEgA',
   'x-amz-request-id': '2f24fc24-86db-4201-a974-169473d57f32',
   'x-amz-id-2': 's9lzHYrFp76ZVxRcpX9+5cjAnEH2ROuNkd2BHfIa6UkFVdtjf5mKR3/eTPFvsiP/XV/VLi31234=',
   'connection': 'close',
   'content-length': '0',
   'date': 'Sat, 13 Apr 2024 08:56:24 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0},
 'ETag': '"d2435bcd89aa4592e75f8555dddcffd4"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'OFZ-Ddzgf6apYdhL9tiEgA'}

In [30]:
list_objects(bucket_name_news)

,Key,LastModified,ETag,Size,StorageClass,Owner.DisplayName,Owner.ID
0,inventory.csv,2024-04-13 08:37:17+00:00,"""d41d8cd98f00b204e9800998ecf8427e""",0,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
1,photo_2024-04-11_13-39-56.jpg,2024-04-13 08:40:06+00:00,"""546a332e7b711e1c2ea615b2824aa593""",89763,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
2,s3://news/Business/dt=20240413/data_Business_n...,2024-04-13 08:36:35+00:00,"""dde88c71110a9c2702dea3f0664104ad""",44450,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
3,s3://news/Entertainment/dt=20240413/data_Enter...,2024-04-13 08:36:35+00:00,"""6208fcf0acc5fd4194e104aa75d6f0ab""",20884,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
4,s3://news/Health/dt=20240413/data_Health_news.csv,2024-04-13 08:36:35+00:00,"""7ee0751cd44a6fd2288d9e60c437ed18""",47526,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
5,s3://news/Science/dt=20240413/data_Science_new...,2024-04-13 08:36:35+00:00,"""ddce3cced04a91878d95f4be0c14d833""",43407,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
6,s3://news/Sports/dt=20240413/data_Sports_news.csv,2024-04-13 08:36:35+00:00,"""83c357ae6a767471f5497f2428853249""",30624,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
7,s3://news/Technology/dt=20240413/data_Technolo...,2024-04-13 08:36:35+00:00,"""3ef519c3d6d503512ffb055a46188a3f""",31218,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
8,s3://news/US/dt=20240413/data_US_news.csv,2024-04-13 08:36:35+00:00,"""59c27a6bc7bba8a7de2e2d0dd6bc6871""",37360,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...
9,s3://news/World/dt=20240413/data_World_news.csv,2024-04-13 08:36:36+00:00,"""17472b76f0644abc5726db0ae4b6dfb4""",30499,STANDARD,webfile,75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c07...


### 🗑️ Create a static site using s3 bucket

In this section, we need to utilize a different command, which requires prior installation of the `awscli-local` tool specifically designed for use with **LocalStack**.

The `awscli-local` tool facilitates developers in seamlessly engaging with the **LocalStack** instance, because you can automatically redirecting commands to local endpoints instead of real AWS endpoints.

In [32]:
# install awslocal to use the cli to interact with localstack
!pip3.12 install awscli-local
!pip3.12 install awscli

   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.4 MB 1.3 MB/s eta 0:00:04
   - -------------------------------------- 0.1/4.4 MB 1.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.2/4.4 MB 1.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/4.4 MB 2.0 MB/s eta 0:00:03
   ---- ----------------------------------- 0.5/4.4 MB 2.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.7/4.4 MB 2.5 MB/s eta 0:00:02
   ------- -------------------------------- 0.9/4.4 MB 2.7 MB/s eta 0:00:02
   --------- ------------------------------ 1.0/4.4 MB 2.8 MB/s eta 0:00:02
   ----------- ---------------------------- 1.2/4.4 MB 3.1 MB/s eta 0:00:02
   ------------ --------------------------- 1.4/4.4 MB 3.1 MB/s eta 0:00:01
   -------------- ------------------------- 1.6/4.4 MB 3.1 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.4 MB 3.3 MB/s eta 0:00:01
   ----------------

In [33]:
# the following command creates a static website in s3
!awslocal s3api create-bucket --bucket docs-web
# add the website configuration
!awslocal s3 website s3://docs-web/ --index-document index.html --error-document error.html
# syncronize the static site with the s3 bucket
!awslocal s3 sync static-site s3://docs-web

#------------------------------------------------------------------------------------------

# If you are using localstack, you can access the website using the following url
#  http://docs-web.s3-website.localhost.localstack.cloud:4566/


#------------------------------------------------------------------------------------------

{
    "Location": "/docs-web"
}
Completed 698 Bytes/70.0 KiB (12.7 KiB/s) with 3 file(s) remaining
upload: static-site\error.html to s3://docs-web/error.html        
Completed 698 Bytes/70.0 KiB (12.7 KiB/s) with 2 file(s) remaining
Completed 1.8 KiB/70.0 KiB (31.4 KiB/s) with 2 file(s) remaining  
upload: static-site\index.html to s3://docs-web/index.html        
Completed 1.8 KiB/70.0 KiB (31.4 KiB/s) with 1 file(s) remaining
Completed 70.0 KiB/70.0 KiB (1.2 MiB/s) with 1 file(s) remaining
upload: static-site\img\img_localstack.png to s3://docs-web/img/img_localstack.png


Url Site: http://docs-web.s3-website.localhost.localstack.cloud:4566/

![](img/localstack-static-site.png)

# 📚 References
If you want to learn...

* [AWS Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#installation)
* [LocalStack](https://docs.localstack.cloud/overview/)
* [API:OkSurf News](https://ok.surf/#endpoints)